In [2]:
!pip install pygame

     |████████████████████████████████| 11.8MB 5.2MB/s 


In [3]:
#DQN
import pygame
import time
import random
from collections import deque
import gym
import numpy as np
import tensorflow as tf #2.0
from tensorflow import keras
from tensorflow.keras import layers
#archive = deque(maxlen=100)
 
pygame.init() #初始化Pygame模組
 
white = (255, 255, 240) #活動區域
black = (0, 139, 69) #蛇的顏色
red = (178, 34, 34) #文字顏色
blue = (25, 25, 112) #食物顏色
 
dis_width = 800 #範圍寬度
dis_height = 600 #範圍高度
 
dis = pygame.display.set_mode((dis_width, dis_height)) #建立遊戲表面
pygame.display.set_caption('Snake Game with DQN') #建立標題
 
clock = pygame.time.Clock()
 
snake_block = 10 #蛇大小
snake_speed = 30 #蛇速度
 
font_style = pygame.font.SysFont(None, 30) #文字風格

rewards = 0
reward = 0
archive = []
target = []
Iteration = 3
gamma = 0.8 #折扣率
epsilon = 0.8  #Epsilon
learning_rate = 0.001
batch_size = 16


 
def message(msg, color):
    mesg = font_style.render(msg, True, color) 
    dis.blit(mesg, [dis_width/3, dis_height/3])
    
#類神經網路    
def NN(archive,target):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(100, activation=tf.nn.relu),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(4, activation=tf.nn.softmax)
    ])
    model.compile(optimizer= 'adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    history = model.fit(archive,target, epochs=1, verbose=1)
    #y_pred = model.predict_classes(x[0:3])
    return model

#動作的編碼與解碼
def encode(x):
    if x == 'a':
        x_output = 0
    elif x == 'b':
        x_output = 1
    elif x == 'c':
        x_output = 2
    elif x == 'd':
        x_output = 3
    return x_output

def decode(x):
    if x == 0:
        x_input = 'a'
    elif x == 1:
        x_input = 'b'
    elif x == 2:
        x_input = 'c'
    elif x == 3:
        x_input = 'd'
    return x_input


food_x = round(random.randrange(0, dis_width - snake_block) / 10.0) * 10.0
food_y = round(random.randrange(0, dis_width - snake_block) / 10.0) * 10.0   
    
def gameLoop():  # creating a function
    a_x = ['a','b','c','d'] #左、右、上、下
    x = random.choice(a_x) 
    
    game_over = False
    game_close = False
 
    x_move = dis_width / 2
    y_move = dis_height / 2
 
    x1_change = 0
    y1_change = 0
 
    #隨機初始產生食物的位置

    #food_x = 350.0 
    #food_y = 310.0
 
    while not game_over:

 
        while game_close == True: #遊戲結束的畫面
            dis.fill(white)
            message("Lost! Enter Q(Exit) or C(Again)", red)
            pygame.display.update()
 
            for event in pygame.event.get(): 
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_q: #選擇Q結束遊戲
                        game_over = True 
                        game_close = False
                    if event.key == pygame.K_c: #選擇C重新遊戲
                        gameLoop()
        
        for event in pygame.event.get(): 
            if event.type == pygame.QUIT:
                game_over = True   
 
        if x_move >= dis_width or x_move < 0 or y_move >= dis_height or y_move < 0: #碰到邊界結束遊戲
            game_close = True
        
       
        #透過DQN移動蛇
        if x == 'a':
            x1_change = -snake_block
            y1_change = 0
        elif x == 'b':
            x1_change = snake_block
            y1_change = 0
        elif x == 'c':
            y1_change = -snake_block
            x1_change = 0
        elif x == 'd':
            y1_change = snake_block
            x1_change = 0

        x_move += x1_change #持續移動
        y_move += y1_change #持續移動
        dis.fill(white)
        pygame.draw.rect(dis, blue, [food_x, food_y, snake_block, snake_block])
        pygame.draw.rect(dis, black, [x_move, y_move, snake_block, snake_block])
        pygame.display.update()
        rewards = reward - 1 #沒吃到-1

        if x_move == food_x and y_move == food_y: #碰到食物就顯示吃到了
            rewards = reward + 1
            print("吃到了！")
        clock.tick(snake_speed)
        
        #將每一次學習進行回饋，並加上epsilon
        print(rewards)
        #q = learning_rate*(reward + gamma)
        archive.append([reward,0])
        number = encode(x)
        target.append([number])
        model = NN(archive,target)
        
        if np.random.rand() <= epsilon:
            a_x = ['a','b','c','d'] #左、右、上、下
            x = random.choice(a_x) 
        else:
            y_pred = model.predict_classes(archive)
            x = decode(random.choice(y_pred))
        print(x)
        #print(model.predict_classes(x))
        #print(archive,target)
        #return archive,target
    pygame.quit()
    quit()
 
gameLoop()

pygame 2.0.1 (SDL 2.0.14, Python 3.7.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


error: ignored